### Variables
We have our system state defined as  - 
* SS: (n, nd, (ch1, st1, r1,m, tbm, sbm), (ch2, st2, r2,m, tb2,m, sb2,m),.. (chn, stn, rn,m, tbn,m, sbn,m))
    * n - total number of available buses
    * m - subscript that denotes route m
    * nd - number of buses deployed
    * chn -  charging state of bus n
    * stn - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * rn,m - the route on which the bus is deployed
    * tbn,m - the previous stop the bus travelled to (travel time)
    * sbn,m - the previous stop which was served by the bus (service time)
        * If tbn= sbn, generate travel time for tbn+1; tbn = tbn+1 if accepted
        * Else if tbn> sbn, generate service time for sbn+1; sbn = sbn+1 if accepted

* t - the clock time/current time
* nrm - number of buses required on route m
* btn - time array for monitoring travel time and service time related to each bus


### Pseudo-code
1. Start at t = 0
2. Generate requirements from route, nd (required number of buses):
    1. Check availability of buses & minimum fuel level/battery capacity required for the routes
    2. Choose number of buses, K, to be deployed based on checks
        1. If K<nd, deploy buses with higher fuel/battery level and recharge/refuel the rest
        2. Else, send buses for recharge/refuel
    3. For each bus k, in {1, 2,.. n}, after selecting a specific route m {stops - 1, 2,... M}:
    4. Initialize j = 0 (Depot_Start)
    5. Generate interarrival times for stop j, update time tbk,j
    6. Generate service time at stop j, update time sbk,j
    7. STOP if j = M; Else update j = j + 1, goto 2.b. 
3. Based on the time we update the System State (SS) and the concerned variables.

### Cases
1. We generate demand for the route at time t,
    Sub-task: Generating series of buses to be deployed
    1. Condition 1: number of buses required <= number of charged/fuelled buses
        1. Deploy buses based on descending order of fuel level/charge
        2. Send buses for refuelling/recharging, if they are below a certain threshold
    2. Condition 2: number of buses required > number of charged/fuelled buses
        1. Send buses for refuelling/recharging
        2. As soon as the bus finishes recharging/refueling, buses get sent to the route
2. Routing of buses:
    1. Route 0: based on type of the bus and refuelling need. Will be a refuelling/ recharging route for the bus
    2. Route 1 to m: Buses will go through the stops till they reach Depot_Stop


## Variables
#### Buses
* bus: ($ch_n, st_n, r_{n,m}, tb_{n,m}, sb_{n,m}$)
    * $ch_n$ - charging state of bus n
    * $st_n$ - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * $r_{n,m}$ - the route on which the bus is deployed
    * $tb_{n,m}$ - the previous stop the bus travelled to (travel time)
    * $sb_{n,m}$ - the previous stop which was served by the bus (service time)
        * If $tb_n$= $sb_n$, generate travel time for $tb_{n+1}$; $tb_n$ = $tb_{n+1}$ if accepted
        * Else if $tb_n$> $sb_n$, generate service time for $sb_{n+1}$; $sb_n$ = $sb_{n+1}$ if accepted
    * Arrival time - route generated arrays
    * Service Time - route generated arrays
    * 

___
### System State New
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill)
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill, route_1_buses, route_2_buses,....)

#### Monitoring System State 

| Time | System State | Bus | Charge | Route | State | Event | Process Time | Demand-Current | Demand-Actual | Demand-Charge |
| ---- | ------------ | --- | ------ | ----- | ----- | ----- | ------------ | -------------- | ------------- | ------------- |
| 0    | (1,1,1,0,0)  | 1   | 50     | '1'   | 1     | 0     | 0            | 0              | 0             | 15            |
| 10.3 | (1,0,1,0,0)  | 1   | 48     | '1'   | 1     | 1     | 10.3         | -              | -             | -             |
| 12.3 | (1,0,1,0,0)  | 1   | 47.5   | '1'   | 1     | 0     | 2.0          | -              | -             | -             |

___

#### Monitoring Bus Deployments

| Time | Demand-Current | Demand-Actual | Demand-Charge | Bus | Charge | Route | Event Array  | Time Array   |
| ---- | -------------- | ------------- | ------------- | --- | ------ | ----- | ------------ | ------------ |
| 0    | 0              | 0             | 15            | 1   | 50     | '1'   | [1, 0, 1]    | [12, 35, 41] |
| 40   | 41             | 40            | 20            | 1   | 35     | '2'   | [1, 0, 1]    | [48, 50, 55] |



In [1]:
import pandas as pd
import numpy as np
import numpy.random as random
from route_functions import *
import matplotlib.pyplot as plt

[nan, nan, nan, nan, nan]

In [20]:
def fleet_simulation(t, T, routes, buses, refuel, running_consumption, service_consumption, 
                     refuel_stations, refuel_consumption, conversion_factor,
                     demand_at, demand_ct, demand_r, demand_c, dct_flag, SS_cols, BD_cols):
    ss_table = pd.DataFrame(columns=SS_cols)
    bd_table = pd.DataFrame(columns=BD_cols)

    # Initial SS update
    time_check = np.inf
    ss_table = SS_update(ss_table, t, buses, dct_flag, np.nan, np.nan)
    
    # previous times for fleet
    prev_time = [np.nan]*len(buses)
    
    # Simulate! Simulate! Simulate!        
    while (t<T)or(time_check!=np.inf):
        print('\n---\nNew Loop')

        time_check = min(next_bus_e(buses)[0], demand_ct[0])
        print('Time check -', time_check)

        # -----
        # Priority One: Updating demands
        if ((demand_ct[0]!=np.inf) and
            (demand_ct[0]==time_check)):
            new_demand = [1 for i in range(len(demand_ct)) if demand_ct[i]==time_check]
            dct_flag = sum(new_demand)

        #print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
        #print('\tdct_flag -', dct_flag)


        #----------
        # Main Simulation
        bus_chk = available_bus(buses, demand_c[0])
        print('Bus Check -', bus_chk)


        #-----
        # Case 1: Sending low-fuel buses to refuel
        refuel_index = unavailable_bus(buses, min(demand_c))
        if (#(t<T) and
            (refuel_index!=-1) and
            (buses_status(buses)[1]<refuel_stations)): 
            # or((dct_flag>0)and(refuel_index!=np.inf)):- might create complications
            dem_ct, dem_at, dem_c = np.nan, np.nan, np.nan
            print('Sending Bus for Refuel')
            buses[refuel_index].assign_route(routes, refuel, t)
            prev_time[refuel_index] = t
            bus_e = 0
            t_updt = t

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, refuel_index)
            bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, refuel_index)

            t = t_updt
            refuel_index = -1


        #-----
        # Case 2: Checking bus availability and deploying buses
        elif ((t<T) and
              (demand_ct[0]==time_check) and
              (demand_ct[0]!=np.inf) and
              (bus_chk!=-1)):

            print('Deploying Bus')
            dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
            dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
            index = available_bus(buses, dem_c)
            buses[index].assign_route(routes, dem_r, t)
            prev_time[index] = t
            t_updt = dem_ct
            bus_e = 0

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
            bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
            print(buses[index].event_arr, buses[index].time_arr)
            dct_flag -= 1
            t = t_updt

            dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
            t_updt = np.nan
            index = np.nan


        #-----
        # Case 3: Checking next bus event and updating SS
        elif ((t<T) and
              (next_bus_e(buses)[0]==time_check) and 
              (next_bus_e(buses)[0]!=np.inf) and 
              ((buses_status(buses)[0]>0) or (buses_status(buses)[1]>0))):

            print('Updating next bus event')
            index = next_bus_e(buses)[2]
            t_updt = buses[index].time_arr.pop(0)
            bus_e = buses[index].event_arr.pop(0)
            diff_t = prev_time[index]
            
            if (bus_e==1):
                mul_c = running_consumption
            elif (bus_e==0)and(buses[index].state==1):
                mul_c = service_consumption
            elif (bus_e==0)and(buses[index].state==0):                       # different for 'refill' and 'recharge' 
                mul_c = refuel_consumption                                   # difference from 90 for refill
            buses[index].charge = buses[index].charge - (t_updt - diff_t)*mul_c*conversion_factor

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
            t = t_updt
            prev_time[index] = t
            
            # if its the last event for the bus, update bus parameters to standstill
            if buses[index].next_e()==np.inf:
                buses[index].state = -1
                buses[index].route = None
                buses[index].time_arr = list()
                buses[index].event_arr = list()
                prev_time[index] = np.nan
            index = np.nan
            t_updt = np.nan
            bus_e = np.nan
        
        
        #-----
        # Case 4: Jumping to next demand 
        elif ((t<T) and
              (demand_ct[0]==time_check) and 
              (demand_ct[0]!=np.inf) and 
              (bus_chk==-1)):
            t = demand_ct[dct_flag]
        
              
        #-----
        # Case 5: All demands are completed within the timeframe 
        elif ((t<T) and
              (time_check==np.inf)):

            print('Jumping to EOD')
            t_updt = T

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
            t = t_updt    


        #-----
        # Case 6: Demands still exist beyond timeframe and need to be met
        elif ((t>T) and
              (demand_ct[0]==time_check) and
              (demand_ct[0]!=np.inf) and
              (bus_chk!=-1)):

            print('Deploying Bus beyond timeframe')
            dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
            dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
            index = available_bus(buses, dem_c)
            buses[index].assign_route(routes, dem_r, t)
            prev_time[index] = t
            t_updt = dem_ct
            bus_e = 0

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
            bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
            dct_flag -= 1
            t = t_updt

            dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
            t_updt = np.nan
            index = np.nan


        #-----
        # Case 7: Going beyond timeframe, checking deployed buses and updating SS
        elif (((t>T) and
              (next_bus_e(buses)[0]==time_check) and
              (next_bus_e(buses)[0]!=np.inf)) or
              ((t>T) and
              (demand_ct[0]==time_check) and 
              (demand_ct[0]!=np.inf) and 
              (bus_chk==-1))):
            
            print('Updating next bus event beyond timeframe')
            index = next_bus_e(buses)[2]
            t_updt = buses[index].time_arr.pop(0)
            bus_e = buses[index].event_arr.pop(0)
            diff_t = prev_time[index]
            
            if (bus_e==1):
                mul_c = running_consumption
            elif (bus_e==0)and(buses[index].state==1):
                mul_c = service_consumption
            elif (bus_e==0)and(buses[index].state==0):                       # different for 'refill' and 'recharge' 
                mul_c = refuel_consumption                                   # difference from 90 for refill
            buses[index].charge = buses[index].charge - (t_updt - diff_t)*mul_c*conversion_factor

            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
            t = t_updt
            prev_time[index] = t
            
            # if its the last event for the bus, update bus parameters to standstill
            if buses[index].next_e()==np.inf:
                buses[index].state = -1
                buses[index].route = None
                buses[index].time_arr = list()
                buses[index].event_arr = list()
                prev_time[index] = np.nan
            index = np.nan
            t_updt = np.nan
            bus_e = np.nan

        #-----
        # Case 8: All demands are completed outside the timeframe 
        elif ((t>T) and
              (time_check==np.inf)):

            print('Jumping to EOD')
            t_updt = np.inf        
            ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
            t = t_updt    



        #print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
        #print('\tdct_flag -', dct_flag)

        # updating the current demand - if demand exists it'll automatically be updated
        for k in range(dct_flag):
            demand_ct[k] = t

        #print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
        #print('\tdct_flag -', dct_flag)
        #print('t -', t)
    
    return ss_table, bd_table

In [31]:
# intializing route table
routes = pd.read_excel('Model_Parameters.xlsx', 'Routes')

# customizable parameters
#average_bus_speed = 10
running_consumption = 0.09             # fuel(lts.)/kWh per minute
service_consumption = 0.075 
refuel_stations = 2
refuel_consumption = -30
tank_size = 200
conversion_factor = 100/tank_size


# tank level/charge of buses
level_mean = 50       # normal distribution.. can be changed
level_std = 2.5
refuel = 'refill'     # 'recharge'

# setting up initial values for simulation
n_buses = 6                                # number of buses
n_routes = 2                               # number of routes
buses = [bus(level_mean, level_std) for i in range(n_buses)]    # fleet of buses
SimTime = 720

# updating demands
demand_at, demand_r, demand_c = gen_demands(n_routes, SimTime)
demand_ct = demand_at.copy()               # demand current time and demand actual time
dct_flag = 0                               # demand at current time

'''# -----
#### creating fake data & demands
t = 0                                      # start time of simulation
T = 60
buses = [bus(5, 5) for i in range(1)]
demand_at = [0, 40, np.inf]
demand_r = ['1', '2', None]
demand_c = [15, 20, np.inf]
demand_ct = demand_at.copy()         # demand current time and demand actual time
dct_flag = 0                  # demand at current time'''

SS_cols = ['Time', 'System_State', 'Bus', 'Charge', 'Route', 
           'State', 'Event', 'Process_Time', 'Demand_Current', 
           'Demand_Actual', 'Demand_Charge']
BD_cols = ['Time', 'Demand_Current', 'Demand_Actual', 'Demand_Charge', 
           'Bus', 'Charge', 'Route', 'Event_Array', 'Time_Array']
  

[  0  60 120 180 240 300 360 420 480 540 600 660]
[ 2.  5.  5.  5.  2. -0. -0.  1.  3.  5.  5.  4.]
[2. 2. 3. 3. 4. 5. 5. 5. 5. 4. 3. 2.]


In [32]:
replicates = 10
ss_table = pd.DataFrame(columns=SS_cols+['Replication'])
bd_table = pd.DataFrame(columns=BD_cols+['Replication'])

for i in range(replicates):
    print('\n\n-----\n-----\nNEW SIMULATION')
    # intializing required variabes
    buses = [bus(level_mean, level_std) for i in range(n_buses)]    # fleet of buses
    t = 0                                      # start time of simulation
    T = SimTime

    # updating demands
    demand_at, demand_r, demand_c = gen_demands(n_routes, T)
    demand_ct = demand_at.copy()         # demand current time and demand actual time
    dct_flag = 0                         # demand at current time

    ss_tab, bd_tab = fleet_simulation(t, T, routes, buses, refuel, running_consumption, 
                                      service_consumption, refuel_stations, refuel_consumption, conversion_factor,
                                      demand_at, demand_ct, demand_r, demand_c, dct_flag, SS_cols, BD_cols)
    ss_tab['Replication'] = i+1
    bd_tab['Replication'] = i+1
    ss_table = ss_table.append(ss_tab, ignore_index=True)
    bd_table = bd_table.append(bd_tab, ignore_index=True)



-----
-----
NEW SIMULATION
[  0  60 120 180 240 300 360 420 480 540 600 660]
[ 2.  5.  5.  5.  2. -0. -0.  1.  3.  5.  5.  4.]
[2. 2. 3. 3. 4. 5. 5. 5. 5. 4. 3. 2.]

---
New Loop
Time check - 0.0
Bus Check - 1
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [2.2758, 2.4036, 9.9441, 10.3902, 15.1992, 16.0242, 21.886]

---
New Loop
Time check - 0.0
Bus Check - 5
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [2.5633, 3.2588, 17.5798, 19.1839, 33.3803]

---
New Loop
Time check - 2.2758
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 2.4036
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 2.5633
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 3.2588
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 9.9441
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 10.3902
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 15.1992
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 16.0242
Bus Check - 2
Updating

C:\Users\krishrao\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)



---
New Loop
Time check - 17.5798
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 19.1839
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 21.886
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 30.0
Bus Check - 2
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [27.3752, 27.8446, 45.8653, 46.4241, 51.2307, 52.8008, 64.2549]

---
New Loop
Time check - 27.3752
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 27.3752
Bus Check - 0
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [33.7836, 33.9226, 47.7657, 48.0567, 59.3632]

---
New Loop
Time check - 27.8446
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 33.3803
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 33.7836
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 33.9226
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 45.8653
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 46.4241
Bus Check - 1
Updating next 


---
New Loop
Time check - 170.6509
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 170.6901
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 170.9245
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 172.3113
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 175.0086
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 175.7411
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 176.5131
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 178.4352
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 180.0
Bus Check - 4
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [183.7264, 183.738, 195.2788, 195.2978, 199.4114, 199.4401, 206.1828]

---
New Loop
Time check - 180.0
Bus Check - 5
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [181.9326, 182.0409, 198.6725, 199.6367, 204.9789]

---
New Loop
Time check - 180.2683
Bus Check - -1
Updating next bus event

---
New Loop
Time check - 180.8489
Bus Ch

[1.0, 0.0, 1.0, 0.0, 1.0] [302.8842, 303.1957, 329.5186, 331.7852, 337.319]

---
New Loop
Time check - 302.2208
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 302.8842
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 303.1957
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 308.8222
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 312.0
Bus Check - 3
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [316.3714, 316.4494, 334.0301, 334.3845, 337.3476]

---
New Loop
Time check - 316.3714
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 316.4494
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 324.0
Bus Check - 5
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [320.8894, 320.9787, 344.889, 345.2831, 350.4052]

---
New Loop
Time check - 320.8894
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 320.9787
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 329.5186
Bus Check - 4
Updatin

...


---
New Loop
Time check - 63.0399
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 65.1317
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 66.3466
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 72.0
Bus Check - 0
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [67.6182, 68.8752, 85.0253, 87.1021, 92.8632, 93.1857, 97.9701]

---
New Loop
Time check - 67.6182
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 68.8752
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 77.3201
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 79.3852
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 81.386
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 81.7374
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 84.0
Bus Check - 3
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [85.2235, 85.6076, 101.3973, 102.756, 110.8069, 111.5075, 124.1342]

---
New Loop
Time check - 85.0253
Bus Check - 2


---
New Loop
Time check - 186.557
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 186.7301
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 186.803
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 187.0243
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 188.8787
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 192.0
Bus Check - 1
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [193.2722, 193.5154, 204.5611, 207.3565, 219.1151, 220.532, 227.0659]

---
New Loop
Time check - 193.2722
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 193.5154
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 193.8424
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 194.5315
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 197.949
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 198.0918
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 200.0
Bus


---
New Loop
Time check - 328.4188
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 333.0534
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 336.0
Bus Check - 3
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [336.0441, 336.3161, 346.4489, 349.4592, 356.5566]

---
New Loop
Time check - 336.0441
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 336.3161
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 337.4865
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 338.1237
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 346.4489
Bus Check - 5
Updating next bus event

---
New Loop
Time check - 348.0
Bus Check - 5
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [349.4938, 349.6833, 365.284, 370.4847, 374.3024]

---
New Loop
Time check - 348.5316
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 349.4592
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 349.4938
Bus Check - 0
Updating 


---
New Loop
Time check - 510.1332
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 510.3778
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 511.7741
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 511.9802
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 512.9663
Bus Check - 1
Updating next bus event

---
New Loop
Time check - 516.0
Bus Check - 1
Deploying Bus
[1.0, 0.0, 1.0, 0.0, 1.0] [523.3941, 523.4359, 539.6266, 540.3242, 552.8004]

---
New Loop
Time check - 519.8245
Bus Check - 0
Updating next bus event

---
New Loop
Time check - 520.0
Bus Check - 0
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [525.6999, 525.9378, 532.8546, 533.7796, 539.8992, 540.4981, 544.1666]

---
New Loop
Time check - 523.3941
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 523.4359
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 525.6999
Bus Check - 4
Updating next bus event

---
New Loop
Time check - 525.9349
Bus Ch


---
New Loop
Time check - 621.7897
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 622.2283
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 623.9369
Bus Check - 3
Updating next bus event

---
New Loop
Time check - 624.0
Bus Check - 3
Deploying Bus
[1, 0, 1, 0, 1, 0, 1] [629.6413, 629.7153, 639.4982, 640.2335, 647.8145, 648.3309, 661.535]

---
New Loop
Time check - 624.5837
Bus Check - -1
Updating next bus event

---
New Loop
Time check - 627.4604
Bus Check - -1
Updating next bus event

---
New Loop
Time check - 628.7372
Bus Check - -1
Updating next bus event

---
New Loop
Time check - 629.0685
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 629.6413
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 629.7153
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 632.6604
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 633.4085
Bus Check - 2
Updating next bus event

---
New Loop
Time check - 634

In [33]:
ss_tab

,Time,System_State,Bus,Charge,Route,State,Event,Process_Time,Demand_Current,Demand_Actual,Demand_Charge,Replication
0,NaN,"[6, 0, 0, 0, 6]",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,10
1,0.0000,"[6, 2, 1, 0, 5]",4,52.919948,1,1,0,0.0000,0.0,0.0,25.0,10
2,0.0000,"[6, 1, 2, 0, 4]",5,52.704804,2,1,0,0.0000,0.0,0.0,25.0,10
3,2.2234,"[6, 0, 2, 0, 4]",4,52.819895,1,1,1,2.2234,NaN,NaN,NaN,10
4,2.4071,"[6, 0, 2, 0, 4]",4,52.813007,1,1,0,0.1837,NaN,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...,...
551,722.0753,"[6, 0, 2, 0, 4]",2,31.183422,1,1,1,0.5661,NaN,NaN,NaN,10
552,724.3705,"[6, 0, 1, 0, 5]",3,31.362847,1,1,1,2.2952,NaN,NaN,NaN,10
553,724.6023,"[6, 0, 1, 0, 5]",3,31.354154,1,1,0,0.2318,NaN,NaN,NaN,10
554,733.6426,"[6, 0, 1, 0, 5]",3,30.947341,1,1,1,9.0403,NaN,NaN,NaN,10


In [34]:
bd_tab['Charge']

0     52.919948
1     52.704804
2     51.645610
3     51.645028
4     51.123653
        ...    
75    32.747533
76    32.921444
77    32.760023
78    32.724022
79    32.353435
Name: Charge, Length: 80, dtype: float64

In [35]:
ss_table

,Time,System_State,Bus,Charge,Route,State,Event,Process_Time,Demand_Current,Demand_Actual,Demand_Charge,Replication
0,NaN,"[6, 0, 0, 0, 6]",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0000,"[6, 2, 1, 0, 5]",2,52.706888,1,1,0,0.0000,0.0,0.0,25.0,1
2,0.0000,"[6, 1, 2, 0, 4]",6,52.046247,2,1,0,0.0000,0.0,0.0,25.0,1
3,2.2758,"[6, 0, 2, 0, 4]",2,52.604477,1,1,1,2.2758,NaN,NaN,NaN,1
4,2.4036,"[6, 0, 2, 0, 4]",2,52.599684,1,1,0,0.1278,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5555,722.0753,"[6, 0, 2, 0, 4]",2,31.183422,1,1,1,0.5661,NaN,NaN,NaN,10
5556,724.3705,"[6, 0, 1, 0, 5]",3,31.362847,1,1,1,2.2952,NaN,NaN,NaN,10
5557,724.6023,"[6, 0, 1, 0, 5]",3,31.354154,1,1,0,0.2318,NaN,NaN,NaN,10
5558,733.6426,"[6, 0, 1, 0, 5]",3,30.947341,1,1,1,9.0403,NaN,NaN,NaN,10


In [39]:
bd_table

,Time,Demand_Current,Demand_Actual,Demand_Charge,Bus,Charge,Route,Event_Array,Time_Array,Replication,State
0,0.0000,0.0000,0.0,25.0,2,52.706888,1,"[1, 0, 1, 0, 1, 0, 1]","[2.2758, 2.4036, 9.9441, 10.3902, 15.1992, 16....",1,1.0
1,0.0000,0.0000,0.0,25.0,6,52.046247,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[2.5633, 3.2588, 17.5798, 19.1839, 33.3803]",1,1.0
2,30.0000,30.0000,30.0,25.0,3,52.013051,1,"[1, 0, 1, 0, 1, 0, 1]","[27.3752, 27.8446, 45.8653, 46.4241, 51.2307, ...",1,1.0
3,27.3752,27.3752,30.0,25.0,1,51.959271,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[33.7836, 33.9226, 47.7657, 48.0567, 59.3632]",1,1.0
4,60.0000,60.0000,60.0,25.0,2,51.732509,1,"[1, 0, 1, 0, 1, 0, 1]","[64.9402, 65.3174, 85.7379, 86.1053, 98.0572, ...",1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
795,660.0000,660.0000,660.0,25.0,6,32.747533,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[666.3323, 666.666, 679.8312, 679.9682, 685.0299]",10,1.0
796,675.0000,675.0000,675.0,25.0,4,32.921444,1,"[1, 0, 1, 0, 1, 0, 1]","[675.8022, 675.9726, 701.4206, 702.3372, 712.0...",10,1.0
797,690.0000,690.0000,690.0,25.0,2,32.760023,1,"[1, 0, 1, 0, 1, 0, 1]","[694.0379, 694.9039, 706.3815, 707.2046, 709.5...",10,1.0
798,690.0000,690.0000,690.0,25.0,1,32.724022,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[694.0728, 695.3784, 707.0469, 709.1268, 715.9...",10,1.0


___
## Model Analysis

In [27]:
# delay analysis on deployments
num_delay = sum(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])>0)
total_delay = sum(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])))
avg_delay = np.mean(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])))

num_delay, round(total_delay, 4), round(total_delay, 4)/num_delay, round(avg_delay, 4)

(13, 10.0836, 0.7756615384615385, 0.0126)

In [28]:
sum(bd_table['Route']!='refill')

800

In [29]:
sum(bd_table['Route']=='refill')

0

In [30]:
# Charge distribution
ss_table['Charge'].min(), ss_table['Charge'].max()

(39.88665101713693, 57.51577553732704)

___
## Testing Area

In [12]:
demand_at, demand_r, demand_c = gen_demands(2, 240)
np.array(demand_at)

[  0  60 120 180]
[2. 5. 5. 5.]
[2. 2. 3. 3.]


array([  0.,   0.,  30.,  30.,  60.,  60.,  72.,  84.,  90.,  96., 108.,
       120., 120., 132., 140., 144., 156., 160., 168., 180., 180., 192.,
       200., 204., 216., 220., 228.,  inf])

In [13]:
# testing for demand creation and updation

#demand = 0
arr = [0, 1, 0, 1, 2, 3]
test = 5
new_demand = [1 for i in range(len(arr)) if arr[i]==test]
demand = demand + sum(new_demand)
demand

NameError: name 'demand' is not defined

In [ ]:
routes[routes.columns[routes.columns.str.match('route_1')]]

In [ ]:
print(route_time(routes, 'recharge', 0))
print(route_time(routes, 'refill', 0))
print(route_time(routes, '2', 0))

In [ ]:
n_buses = 4
buses = [bus(50, 5) for i in range(n_buses)]
t = 10
buses[0].assign_route(routes, 'refill', t)
buses[1].assign_route(routes, '1', t)
buses[2].assign_route(routes, '1', t)
print(buses[0].time_arr)
print(buses[0].event_arr)
buses_status(buses)

In [ ]:
print(buses[0].next_t())
print(buses[0].next_e())
print(buses[0].time_arr.pop(0))
print(buses[0].event_arr.pop(0))
print(buses[0].next_t())
print(buses[0].next_e())

In [ ]:
#### ASSUMPTION: buses are prioritized based on their index

def next_bus_e(buses):
    min_t = np.inf
    index = None
    for i in range(len(buses)):
        print(buses[i].next_t())
        if (buses[i].next_t()<min_t):
            min_t = buses[i].next_t()
            index = i
    return min_t, buses[i].next_e(), index

# testing 
buses = [bus(50, 5) for i in range(3)]
buses[0].assign_route(routes, '1', 0)
buses[1].assign_route(routes, '1', 0)
buses[2].assign_route(routes, '1', 0)
print(buses[0].time_arr)
print(buses[0].event_arr)
print(buses[1].time_arr)
print(buses[1].event_arr)
print(buses[2].time_arr)
print(buses[2].event_arr)
next_bus_e(buses)

def available_bus(buses, dem_charge):
    print(buses)
    b_charges = [buses[i].charge for i in range(len(buses)) if buses[i].state==-1]
    b_index = [i for i in range(len(buses)) if buses[i].state==-1]
    index = np.nan
    if len(b_charges)>0:
        if (max(b_charges)>dem_charge):
            for i in range(len(b_charges)):
                if max(b_charges)==b_charges[i]:
                    index = b_index[i]
    return index

check = bus(20, 4)
buses = [bus(25, 5) for i in range(1)]
buses[0].state = 1
available_bus(buses, 20)

#### Testing for route demands

In [ ]:
# route demand functions
array_t = np.array(range(0, 12*60+1, 60))

a, b, c, d = 3, 2, 0, 75
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='r', label='bus_1', marker='o')

a, b, c, d = 2, 3, -180, 120
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='b', label='bus_2', marker='o')

a, b, c, d = 3, 3, 0, 45
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='g', label='bus_3', marker='o')

plt.xlabel('Time')
plt.ylabel('Buses')
plt.legend()
plt.show()